In [1]:
import numpy as np
import re
import json
import pandas as pd
import warnings
import csv
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
%load_ext autoreload
%autoreload 2
import label_improve as li

In [3]:
# Loading the data 
dataset_name = "amazon31"
idx_to_label = json.load(open(f"label.json"))
label_to_idx = {l:i for i,l in idx_to_label.items()}
test_df = li.data_to_df(json.load(open(f"test.json", "r")))

In [4]:
test_df = test_df.sample(250, random_state=205)
eval_df = test_df.drop(columns=[ 'label', 'weak_labels'])

In [5]:
eval_df.to_csv(f'./{dataset_name}_eval.csv')

In [6]:
from openai import OpenAI
import os
import sys

API_KEY = ''


model_id = 'gpt-4o-2024-08-06' # replace with your model ID

os.environ["OPENAI_API_KEY"] = API_KEY

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [7]:
import openai
import pandas as pd

# Load your CSV file
df = pd.read_csv(f'{dataset_name}_eval.csv')


# Define the categories
categories = {
    "0": "Apparel",
    "1": "Automotive",
    "2": "Baby",
    "3": "Books",
    "4": "Camera",
    "5": "Electronics",
    "6": "Gift Card",
    "7": "Grocery",
    "8": "Health & Personal Care",
    "9": "Home",
    "10": "Jewelry",
    "11": "Kitchen",
    "12": "Lawn and Garden",
    "13": "Luggage",
    "14": "Major Appliances",
    "15": "Mobile_Apps",
    "16": "Music",
    "17": "Musical Instruments",
    "18": "Office Products",
    "19": "Outdoors",
    "20": "PC",
    "21": "Pet Products",
    "22": "Shoes",
    "23": "Software",
    "24": "Sports",
    "25": "Tools",
    "26": "Toys",
    "27": "Video",
    "28": "Video Games",
    "29": "Watches",
    "30": "Wireless"
}
# Function to classify text
def classify_text(text):
    response = client.chat.completions.create(
        model=model_id,
        messages=[
            {"role": "system", "content": "You are an expert in Product Categorization, and you are good at classification."},
            {"role": "user", "content": f"Classify the following text into one of the categories: {categories}. Text: {text}, also provide the label number at the end of the sentences, with the exact sentence 'The label is : $number'"}
        ]
    )
    classification = response.choices[0].message.content
    return classification.strip()



# Apply the function to each line of text
df['Classification'] = df['text'].apply(classify_text)

# Save the results to a new CSV file
df.to_csv('classified_file.csv', index=False)


In [9]:
# Extract the label number at the end of the 'Classification' column
df['Classification'] = df['Classification'].str.extract(r'The label is\s*:\s*(\d+)')
 
df.to_csv('classified_file2.csv', index=False)


In [10]:
predictions_df = pd.read_csv('classified_file2.csv')

true_labels_df =  test_df

merged_df = pd.merge(predictions_df, true_labels_df, on='text')

merged_df['correct'] = merged_df['Classification'] == merged_df['label']

accuracy = merged_df['correct'].mean()
print(f"Accuracy: {accuracy:.2%}")

print(merged_df.head())

merged_df.to_csv('comparison_with_true_labels.csv', index=False)


Accuracy: 78.80%
   Unnamed: 0                                               text  \
0        2870  VideoSecu Microphone Audio Pickup Device Sound...   
1       14215  CONAIR SOUND THERAPY W/N One big problem The s...   
2       11734  Tomee SNES USB Controller for PC/ Mac Perfect....   
3        7028  Casio Men's W201-1AV Chronograph Watch Reliabl...   
4       10273  HoMedics PA-MH Therapist Select Percussion Mas...   

   Classification  label                                        weak_labels  \
0             5.0      4  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...   
1             8.0      8  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...   
2            20.0     28  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...   
3            29.0     29  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...   
4             8.0      8  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...   

   correct  
0    False  
1     True  
2    False  
3     True  
4     True  
